![Astrofisica Computacional](../../../logo.png)

---
## 08. Introduction to `AstroPy`. Coordinates and Tables


Eduard Larrañaga (ealarranaga@unal.edu.co)

---

### About this notebook

In this notebook we present an introduction to the use coordinates and tables in `astropy`.

---

In [ ]:
%matplotlib inline  
import matplotlib.pyplot as plt
import numpy as np
from astropy import units as u

## 1. Coordinates

The module [astropy.coordinates](http://docs.astropy.org/en/stable/coordinates/) provides a framework to handle sky positions in various coordinate systems and transformations between them. The basic class to handle sky coordinates is [SkyCoord](http://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html),

In [ ]:
from astropy.coordinates import SkyCoord

We can define the position angle for longitude and latitude for a particular source, together with a keyword specifying a coordinate frame. For example, using the International Celestial Reference System 'icrs' and decimal degrees, we define the position of the crab nebula as

In [ ]:
position_crab = SkyCoord(83.63 * u.deg,  22.01 * u.deg, frame='icrs')
position_crab

It is also possible to use `lists`, `arrays` or even `Quantities` to define the coordinates,

In [ ]:
positions = SkyCoord([345., 234.3] * u.deg,  [-0.1, 0.2] * u.deg, frame='galactic')
positions

An interesting option is to define the angular position using strings with the `'hms'` and `'dms'` notation:

In [ ]:
position_crab = SkyCoord('5h34m31.97s', '22d0m52.10s', frame='icrs')
position_crab

Alternatively, we can use the argument `unit`,

In [ ]:
position_crab = SkyCoord('5:34:31.97', '22:0:52.10',
                         unit=(u.hour, u.deg), frame='icrs')
position_crab

### 1.1. Catalogues

A very convenient and easy way to get the coordinates of a particular source is by using the [Sesame](http://cds.u-strasbg.fr/cgi-bin/Sesame) database with the command `SkyCoord.from_name()`:

In [ ]:
positionCrab = SkyCoord.from_name('Crab')
positionCrab

To access the longitude and latitud angles individually we use the attributes `.lon`and `.lat`

In [ ]:
position_crab.data.lon

In [ ]:
position_crab.data.lat

### 1.2. Transformation between cordinate systems

In order to transform the coordinates from one coordinate system to another system we can use the command `SkyCoord.transform_to()`,

In [ ]:
position_crab

In [ ]:
position_crab_galactic = position_crab.transform_to('galactic')
position_crab_galactic

It is also possible to use the attributes `.galactic` or `.icrs` to perform the transformation:

In [ ]:
position_crab.galactic

In [ ]:
position_crab_galactic.icrs

### 1.3. Measuring distances between positions in the sky

The angular distance between two [SkyCoord](http://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html) objects, can be found using the method [SkyCoord.separation()](http://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html#astropy.coordinates.SkyCoord.separation).

For example, consider the source Sagittarius A* (Sgr A*), at the center of the Milky Way,

In [ ]:
#position_saga = SkyCoord.from_name('Sag A*')
position_SgrA = SkyCoord(0 * u.deg, 0 * u.deg, frame='galactic')
position_SgrA

In [ ]:
position_crab

The distance from the Crab nebula to Sgr A* is

In [ ]:
position_crab.separation(position_SgrA)

The inverse proble is also possible. In this case we want to compute a new position in the sky based on a given offset and position angle. For example, from the Crab nebula we can calculate

In [ ]:
position_crab

In [ ]:
position_crab.directional_offset_by(separation=1 * u.deg, 
                                    position_angle=0 * u.deg)

### 1.4. ALT - AZ coordinates

When planning observations, it is convenient to transform the sky coordinates into a position in the horizontal coordinate system, given a location on earth and a time. We will use the functions [astropy.coordinates.Earthlocation](https://docs.astropy.org/en/stable/api/astropy.coordinates.EarthLocation.html) and [astropy.coordinates.AltAz](https://docs.astropy.org/en/stable/api/astropy.coordinates.AltAz.html),

In [ ]:
from astropy.coordinates import EarthLocation, AltAz
from astropy.time import Time

The location of Bogotá, Colombia, is

In [ ]:
Bogota = EarthLocation(lat=4.7110 * u.deg, lon=-74.0721 * u.deg)
Bogota.geodetic

and the local time is calculated using the [Time](http://docs.astropy.org/en/stable/api/astropy.time.Time.html) object:

In [ ]:
now = Time.now()
print(now)

Now, we define a horizontal coordinate system using the [AltAz]([docs.astropy.org/en/stable/api/astropy.coordinates.AltAz.html) class and use it to convert from the sky coordinate,

In [ ]:
altaz = AltAz(obstime=now, location=Bogota)
crab_altaz = position_crab.transform_to(altaz)
crab_altaz

Note that we obtain the alt-az coordinates and aditional information about the local observation conditions.

## 2. Tables

Another interesting characteristic of `astropy` is the [Table](http://docs.astropy.org/en/stable/api/astropy.io.votable.tree.Table.html) class. This allows to handle data tables and data in .fits files.

In [ ]:
from astropy.table import Table

Table objects are created by

In [ ]:
table = Table()

We add columns to the table like we would add entries to a dictionary (Note the units for the coordinates!!)

In [ ]:
table['Source_Name'] = ['Crab', 'Sag A*', 'Cas A', 'Vela Junior']
table['GalLon'] = [184.5575438, 0, 111.74169477, 266.25914205] * u.deg
table['GalLat'] = [-5.78427369, 0, -2.13544151, -1.21985818] * u.deg
table['Source_Class'] = ['pwn', 'unc', 'snr', 'snr']

By executing the following cell, we get a nicely formatted version of the table printed in the notebook:

In [ ]:
table

### 2.1. Accessing rows and columns

The attribute `.colnames` gives the names of the columns,

In [ ]:
table.colnames

To access individual columns we use their name,

In [ ]:
table['GalLon']

In [ ]:
table[['Source_Name', 'GalLat']]

It is also possible to get the column data as [astropy.units.Quantity](http://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity) using the `.quantity` property,

In [ ]:
table['GalLon'].quantity

Rows can be accessed using numpy indexing,

In [ ]:
table[0:2]

or by using a boolean numpy array for indexing,

In [ ]:
selection = table['Source_Name'] == 'Crab'
table[selection]

More information about indexing can be consulted [here](http://docs.astropy.org/en/stable/table/indexing.html).

### 2.2. Indexing and Grouping

The method `.add_index()` allows to define an "index column" to access rows by the value contained in the index column. For example, we add the index corresponding to the "Source_Name" column,

In [ ]:
table.add_index(colnames="Source_Name")

Now, it is possibleto access a particular row using the using the `.loc[]` syntax (as `pandas`dataframes):

In [ ]:
table.loc["Cas A"]

In [ ]:
table.loc[["Cas A", "Crab"]]

It is also possible to group the rows by a given key column. The groups will be defined by the unique values contained in the column defined as key.

In [ ]:
table_grouped = table.group_by("Source_Class")

for group in table_grouped.groups:
    print(group, "\n")

Each `group` created is again a `Table` object:

In [ ]:
type(group)

### 2.3. Reading / Writing tables to files
Astropy tables can be saved in many formats (for details see [here](http://docs.astropy.org/en/latest/io/unified.html#built-in-table-readers-writers)). 

In [ ]:
table.write('example.fits', overwrite=True, format='fits')

In [ ]:
Table.read('example.fits')

### 2.4. Other operations

Other useful operations when working with Astropy tables.

- Sort by key:

In [ ]:
table.sort('GalLon')

In [ ]:
table

Note that `.sort()` is an "in place operation" on the table, i.e. it changes the actual table.

- To remove a specific row by index:

In [ ]:
table.remove_row(0)
table

- Astropy tables also support row-wise iteration in Python loops:

In [ ]:
for row in table:
    print(row['Source_Name'])

- Another useful feature for quickly inspecting the data contained in the table is the `.show_in_browser()` method:

In [ ]:
table.show_in_browser(jsviewer=True)